<a href="https://colab.research.google.com/github/BartoszJanJerzy/stats/blob/main/stata_KJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#WELCOME!

I (Bartosz Wójtowicz, The John Paul II Catholic University of Lublin) wrote this this code to calculate results in one of master thesis.

---

What is the scehdule?

1 - import libraries

2 - functions used later

3 - import data and screening

4 - research sample

5 - descriptive stats

6 - knowledge questions

7 - main analysis: linear regressions

8 - extra analysis: curvilinear regressions

---

We share this code with you, feel free to use or check it.

#1 - LIBRARY

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr, ttest_ind, kurtosis, skew, shapiro, f_oneway
import statistics
from statsmodels.formula.api import ols
from scipy import stats

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

#2 - FUNCTIONS

#3 - DATA

##Load data

In [3]:
raw_data = pd.read_excel('data.xlsx')
data = raw_data.copy()
data

,sex,age,residence,education,marital_status,relation,relation_type,relation_len,ksp1,ksp2,ksp3,ksp4,ksp5,ksp6,ksp7,ksp8,ksp9,ksp10,ksp11,ksp12,ksp13,ksp14,ksp15,ksp16,ksp17,ksp18,ksp19,ksp20,ksp21,ksp22,ksp23,ksp24,ksp_bezp,ksp_lek,ksp_unik,az1,az2,az3,az4,az5,...,az13,az14,az15,az16,az17,az18,az19,az20,az21,az22,az23,az24,az25,az26,az27,az27_r,az28,az29,az30,az3_r,az6_r,az7_r,az8_r,az10_r.1,az14_r,az17_r,az18_r,az19_r,az21_r,az22_r,az25_r,az27_r.1,az29_r,az30_r,az_wo,bz1,bz2,bz3,bz4,bz5
0,0,24,1,3,2,1,1.0,60.0,6,2,2,7,6,1,7,1,4,7,4,1,7,1,2,5,4,1,7,4,1,7,1,1,53,23,13,3,2,4,2,2,...,4,4,4,3,1,4,3,1,4,5,1,1,5,1,5,1,1,5,5,2,2,2,4,2,2,3,5,2,5,2,5,5,5,5,85,4,3,3,1,5
1,1,24,4,4,1,0,NaN,NaN,6,3,4,5,3,3,5,2,2,5,3,2,5,3,2,6,5,3,6,2,3,5,2,2,43,23,21,3,2,4,2,3,...,5,1,4,2,4,3,3,1,3,3,3,2,3,4,5,1,1,5,3,2,3,5,2,3,1,4,2,3,5,3,4,2,5,5,90,4,3,4,4,3
2,0,23,4,3,2,1,2.0,60.0,6,3,2,6,4,2,7,1,4,6,5,1,6,2,2,6,4,3,6,3,2,5,5,2,48,27,18,4,4,3,3,3,...,4,3,3,3,2,3,3,3,2,4,3,3,2,3,3,3,3,3,3,3,3,3,3,4,2,3,4,3,3,4,3,3,3,3,98,2,2,2,3,2
3,0,24,4,3,1,1,2.0,6.0,5,4,3,5,1,1,6,1,2,5,2,1,6,2,2,3,4,1,4,2,1,4,2,5,38,18,16,4,3,2,1,2,...,1,1,2,2,4,3,2,1,5,5,3,1,4,2,5,1,1,4,5,4,5,2,2,4,5,4,2,3,5,1,5,4,5,5,84,2,5,2,5,2
4,1,23,4,3,2,1,2.0,60.0,7,3,1,7,4,1,7,2,1,7,7,1,7,3,1,7,5,1,7,4,1,6,3,1,55,31,8,1,1,5,1,1,...,4,5,1,2,5,5,5,1,5,5,1,2,5,2,5,1,1,5,5,1,1,2,1,1,2,4,1,1,5,1,4,4,5,5,60,3,1,1,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1,23,4,4,2,1,2.0,4.0,4,5,6,3,4,3,6,7,2,6,5,2,6,6,4,2,5,2,4,5,2,5,5,3,36,42,24,3,3,3,3,3,...,4,1,2,2,2,2,4,2,3,4,3,4,3,3,3,3,4,1,4,3,3,2,3,3,2,4,4,4,4,3,2,3,3,2,90,3,3,2,3,3
76,0,22,4,3,2,1,2.0,36.0,7,6,3,7,5,1,6,6,2,5,5,1,7,3,2,6,5,2,7,5,2,6,4,2,51,39,15,2,2,4,2,3,...,3,4,4,3,3,4,4,2,4,4,2,2,4,3,4,2,2,4,3,2,2,2,2,2,3,3,3,2,4,2,4,3,4,4,78,1,1,1,2,1
77,1,21,4,3,2,1,1.0,36.0,1,2,6,1,5,6,2,1,1,2,1,5,5,1,7,1,1,7,2,7,7,1,1,1,15,19,40,2,2,1,4,3,...,2,4,1,5,5,3,2,1,4,4,2,1,5,5,5,1,1,4,1,5,5,5,4,4,4,1,1,3,5,2,5,1,5,5,95,4,4,1,2,2
78,0,23,1,4,2,1,1.0,45.0,7,2,6,7,5,2,7,1,1,6,3,2,7,1,1,6,7,2,7,1,1,6,1,1,53,21,16,3,2,2,2,3,...,4,4,3,4,3,4,4,1,3,5,1,3,4,2,5,1,1,5,5,4,4,4,4,2,2,2,3,2,5,3,3,4,5,5,89,4,1,1,2,1


In [8]:
print(f'The raw research sample has {len(data)} records')

The raw research sample has 80 records


##Screenning

###Calculate means and stds

In [9]:
mean_bezp = data['ksp_bezp'].mean()
sd_bezp = data['ksp_bezp'].std()

mean_lek = data['ksp_lek'].mean()
sd_lek = data['ksp_lek'].std()

mean_unik = data['ksp_unik'].mean()
sd_unik = data['ksp_unik'].std()

mean_az = data['az_wo'].mean()
sd_az = data['az_wo'].std()

###Make filters

In [12]:
filter_1 = data['ksp_bezp'] > mean_bezp - 3*sd_bezp
filter_2 = data['ksp_bezp'] < mean_bezp + 3*sd_bezp

filter_3 = data['ksp_lek'] > mean_lek - 3*sd_lek
filter_4 = data['ksp_lek'] < mean_lek + 3*sd_lek

filter_5 = data['ksp_unik'] > mean_unik - 3*sd_unik
filter_6 = data['ksp_unik'] < mean_unik + 3*sd_unik

filter_7 = data['az_wo'] > mean_az - 3*sd_az
filter_8 = data['az_wo'] < mean_az + 3*sd_az

###Use filters

In [ ]:
data = data[filter_1]
data = data[filter_2]
data = data[filter_3]
data = data[filter_4]
data = data[filter_5]
data = data[filter_6]
data = data[filter_7]
data = data[filter_8]

In [14]:
print(f'The cleaned research sample has {len(data)} records')

The cleaned research sample has 77 records


#4 - RESEARCH SAMPLE

In [67]:
sex = data['sex'].value_counts(sort=False).to_list()

age_mean = round(data['age'].mean(), 2)
age_std = round(data['age'].std(), 2)
age = [0]

residence = data['residence'].value_counts(sort=False).to_list()
education = data['education'].value_counts(sort=False).to_list()
status = data['marital_status'].value_counts(sort=False).to_list()
relation = data['relation'].value_counts(sort=False).to_list()
relation_type = data['relation_type'].value_counts(sort=False).to_list()

relation_len_mean = round(data['relation_len'].mean(), 2)
relation_len_std = round(data['relation_len'].std(), 2)
relation_len = [0]

Ns = []
for i in [sex, age, residence, education, status, relation, relation_type, relation_len]:
  for j in i:
    Ns.append(round(j, 2))


list_perc = []
for i in Ns:
  list_perc.append(round(i/len(data), 2))

Ns_r = []
for i in Ns:
  if i == 0:
    i = '-'
  Ns_r.append(i)

list_perc_r = []
for i in list_perc:
  if i == 0:
    i = '-'
  else:
    i = str(int(i*100)) + '%' 

  list_perc_r.append(i)

In [68]:
demo_dict = {
    'cat':['Mężczyzna','Kobieta',
           'Wiek',
           'Wieś','Małe miasto (do 50tys)','Średnie miasto (do 100tys)','Duże miasto (pow. 100tys)',
           'Podstawowe','Zasadnicze zawodowe','Średnie','Wyższe',
           'Panna/kawaler','Związek nieformalny','Zamężna/żonaty',
           'Bez związku romantycznego','Związek romantyczny',
           'Pierwszy związek','Kolejny związek',
           'Długość związku (miesiące)'],
    'N':Ns_r,
    '%':list_perc_r,
    'M':['-','-', age_mean, '-','-','-','-','-','-','-','-','-','-','-','-','-','-','-', relation_len_mean],
    'SD':['-','-', age_std, '-','-','-','-','-','-','-','-','-','-','-','-','-','-','-', relation_len_std]
}

demo_df = pd.DataFrame(demo_dict)
demo_df

,cat,N,%,M,SD
0,Mężczyzna,38,49%,-,-
1,Kobieta,39,51%,-,-
2,Wiek,-,-,23.21,1.38
3,Wieś,25,32%,-,-
4,Małe miasto (do 50tys),8,10%,-,-
5,Średnie miasto (do 100tys),11,14%,-,-
6,Duże miasto (pow. 100tys),33,43%,-,-
7,Podstawowe,1,1%,-,-
8,Zasadnicze zawodowe,1,1%,-,-
9,Średnie,40,52%,-,-
